In [1]:
# prerequisites:
# pip install -U swifter[groupby]
import swifter 

import pandas as pd
import numpy as np
import h5py 
import matplotlib.pyplot as plt
from scipy.stats import uniform
from sklearn import linear_model

In [2]:
# HYPER-PARAMETERS 
NOISE_FACTOR = 2  # кол-во шумовых фотонов по отношению к правильным
MINIMUM_FOTONS_NUMBER = 4  # Не рассматриваем случаи с меньшим чем MINIMUM_FOTONS_NUMBER числом фотонов
NUMBER_OF_CASES = 1000 # Кол-во случаев для оценки
BAND_WIDTH = 5 # Половина ширины полоски, в которой считаются фотоны
BAND_ROTATIONS_NUMBER = 100 # число поворотов для определения оптимального угла. минимальный угол поворота = 360гр/BAND_ROTATIONS_NUMBER 

In [3]:
df = pd.read_hdf("./data/Out_fin_pad(200)_x(-1.0,1.0)_E(0.01,10000.0)_eff(0.8)_102.h5")

In [4]:
# генерирует шум
def generate_uniform(df, size):
    size_df = len(df)
    xarr = uniform.rvs(scale = 199, size = size * size_df)
    yarr = uniform.rvs(scale = 199, size = size * size_df)
    return pd.DataFrame({'x_det_pad': xarr.astype(int), 'y_det_pad': yarr.astype(int)})

In [5]:
# определяет, лежит ли точка (px,py) в (верхней) полуплоскости, образуемой прямой, проходящей через (x0,y0), (x1,y1)
def point_belongs_to_semiplane(x0,y0,x1,y1,px,py):
    return np.cross([x1-x0, y1-y0], [px-x0,py-y0]) >= 0

In [6]:
# определяет, лежит ли точка в полосе шириной +-width, проходящую через точку (x0,y0) под углом alpha
def point_belongs_to_band(px,py, x0,y0, alpha, width):
    x1 = x0 + np.cos(alpha)
    y1 = y0 + np.sin(alpha)
    
    width_x = - width * np.sin(alpha)
    width_y = width * np.cos(alpha)
    return (not point_belongs_to_semiplane(x0 + width_x, y0 + width_y,x1 + width_x, y1 + width_y, px, py) and
        point_belongs_to_semiplane(x0 - width_x, y0 - width_y,x1 - width_x, y1 - width_y, px, py) and
        not point_belongs_to_semiplane(x0 - width_x, y0 - width_y, x0 + width_x, y0 + width_y, px, py))

In [7]:
# считает кол-во фотонов, попадающих в полосу шириной +-width, проходящую через точку (x0,y0) под углом alpha
def count_points_in_band(df, x0, y0, alpha, width):         
    x1 = x0 + np.cos(alpha)
    y1 = y0 + np.sin(alpha)
    
    width_x = - width * np.sin(alpha)
    width_y = width * np.cos(alpha)
   
    count = 0   
    for index, row in df.iterrows():
        x = row['x_det_pad']
        y = row['y_det_pad']
        if (not point_belongs_to_semiplane(x0 + width_x, y0 + width_y,x1 + width_x, y1 + width_y, x, y) and
            point_belongs_to_semiplane(x0 - width_x, y0 - width_y,x1 - width_x, y1 - width_y, x, y) and
            not point_belongs_to_semiplane(x0 - width_x, y0 - width_y, x0 + width_x, y0 + width_y, x, y)
           ):
            count += 1
    
    return count

In [8]:
# находит оптимальный угол поворота для отдельного эксперимента путем полного перебора
def find_max_alfa(df, x0, y0, width):
    alpha_max = 0
    cmax = 0
    for alpha in np.arange(-np.pi, np.pi, 2*np.pi/BAND_ROTATIONS_NUMBER):
        c = count_points_in_band(df, x0, y0, alpha, width)
        if c > cmax:
            cmax = c
            alpha_max = alpha
            
    return alpha_max

In [9]:
# оцениваем точность наивного метода для конкретного эксперимента
def evaluate_naive_method(df, width):   
    # не рассматриваем случаи с малым числом фотонов
    if len(df) < MINIMUM_FOTONS_NUMBER:
        return
    
    # находим порождающую частицу
    particle = df.query('Eg == 0')
    # не рассматриваем случаи с двумя и более частицами
    if len(particle) > 1:
        return
    
    # находим координаты этой частицы
    x0 = float(particle.iloc[0]['x_det_pad'])
    y0 = float(particle.iloc[0]['y_det_pad'])
       
    # для оценки точности модели шум добавляем до определения угла поворота
    noise = generate_uniform(df, NOISE_FACTOR)
    df_and_noise = pd.concat([df, noise])
    
    # определяем нужный угол поворота
    alpha_max = find_max_alfa(df_and_noise, x0, y0, width)
    
    # считаем метрики
    TP = 0 # True Positive (TP): you predicted positive, the real value was positive
    TN = 0 # True Negative (TN): you predicted negative, the real value was negative
    FP = 0 # False Positive (FP): you predicted positive, the real value was negative
    FN = 0 # False Positive (FP): you predicted positive, the real value was negative
    for index, row in df_and_noise.iterrows():
        if row['Eg'] == 0: # саму частицу не учитываем
            continue
        if point_belongs_to_band(row['x_det_pad'],row['y_det_pad'], x0,y0, alpha_max, width):
            if np.isnan(row['Num']):
                FP += 1
            else:
                TP += 1
        else:
            if np.isnan(row['Num']):
                TN += 1
            else:
                FN += 1
                  
    #return [TP, TN, FP, FN]
    return pd.DataFrame({'TP': [TP], 'TN': [TN], 'FP': [FP], 'FN': [FN]})

In [10]:
def plot_elements(df, low=0, up=200):
    fig, ax = plt.subplots(figsize=(5,5))
  
    plt.xlim(low,up)
    plt.ylim(low,up)
    particle = df.query('Eg == 0')
    df = df.drop(df[df.Eg == 0].index)
    
    data = df[df.Num.notnull()]
    noise = df[df.Num.isna()]
    
    ax.scatter(data[['x_det_pad']], data[['y_det_pad']], s=2)
    ax.scatter(noise[['x_det_pad']], noise[['y_det_pad']], s=1, facecolor='green')
    ax.scatter(particle[['x_det_pad']], particle[['y_det_pad']], facecolor='red', s=2)
    return fig, ax

In [11]:
# группируем данные по номеру эксперимента, для каждой группы запускаем оценку
evaluation = df.query('Num < @NUMBER_OF_CASES').swifter.groupby("Num").apply(lambda x: evaluate_naive_method(x, BAND_WIDTH))

  0%|          | 0/24 [00:00<?, ?it/s]

2023-12-09 22:51:50,670	INFO worker.py:1673 -- Started a local Ray instance.


In [12]:
evaluation

,,TP,TN,FP,FN
Num,,,,,
0.0,0,5,12,0,0
1.0,0,7,18,2,2
2.0,0,8,17,1,0
3.0,0,3,8,0,0
4.0,0,1,7,1,2
...,...,...,...,...,...
994.0,0,6,14,0,0
995.0,0,3,6,2,0
996.0,0,1,7,3,3


In [13]:
total = evaluation.sum()

In [14]:
# Evaluation metrics: https://www.edlitera.com/blog/posts/evaluating-classification-models

# 1. Доля правильных предсказаний - Accuracy
# Accuracy = (Number of correct predictions) / (Overall number of predictions)

# 2. Точность - Precision
# Precision = (True Positive) / (True Positive + False Positive)

# 3. Полнота - Recall (Sensitivity, True Positive Rate)
# The recall is defined as the number of true positives divided by the sum of true positives and false negatives. 
# It expresses the ability to find all relevant instances in a dataset. 
# Recall measures how good your model is at correctly predicting positive cases. 
# It’s the proportion of actual positive cases which were correctly identified. The equation for recall is:
# Recall = (True Positive) / (True Positive + False Negative)

Accuracy = (total['TP'] + total['TN']) / (total['TP'] + total['TN'] + total['FP'] + total['FN'])
Precision = total['TP'] / (total['TP'] + total['FP'])
Recall = total['TP'] / (total['TP'] + total['FN'])
print('NAIVE METHOD SCORE:')
print('Accuracy:', Accuracy)
print('Precision:', Precision)
print('Recall:', Recall)

NAIVE METHOD SCORE:
Accuracy: 0.8959057259404744
Precision: 0.8448183760683761
Recall: 0.7987373737373737


In [15]:
import ray
ray.shutdown()